In [1]:
import numpy as np
import sys
import os.path
import glob
import random
from scipy.stats import lognorm, norm
from matplotlib import pyplot as plt
import mplhep as hep 
import pyhf
from matplotlib.ticker import ScalarFormatter,AutoMinorLocator
plt.style.use('loca_v_1_5/plot_style.txt')
from loca_v_1_5.loca import *
t0=timer()
plt.style.use(hep.style.CMS)
import yaml
import time


In [2]:
def MonoTau_ATLAS_search(lhco_file, mT_min=0.0):

    process='p p --> τ v'

    search_description='''Recast of ATLAS search arXiv:2002.12223: 
    'SEARCH FOR HEAVY HIGGS BOSONS DECAYING INTO TWO TAU LEPTONS WITH
    THE ATLAS DETECTOR USING pp COLLISIONS AT SQRT(S)=13 TEV'
    Phys. Rev. Lett. 125 (2020) 051801'''

    #...Pre-cuts:
    basic_cuts = {                    
            'e'     : [15.0 , 1e+10 ,  2.47],   #[pt_min, pt_max, abs(eta_max)] 
            'mu'    : [7.0  , 1e+10 ,  2.5],
            'tau'   : [65.0 , 1e+10 ,  2.5],
            'jet'   : [20.0 , 1e+10 ,  2.5],
            'bjet'  : [20.0 , 1e+10 ,  2.5],
            'met'   : [0.0  , 1e+10 ,  1e+10],
            'photon': [0.0  , 1e+10 ,  1e+10] 
             }

    signals=[]; effs=[]; xsecs=[]

    #...Signal region defintion:
    
    Category_τhad = category(
            'nleps = 0',
            'ntaus >= 1',
            'met > 150',
            '|dphi(τ,met)| > 2.4',
            '0.7 < pT/met < 1.3',
            'mT > '+str(mT_min)
             )

    
    #...Observables:
    
    bins=[200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1750, 2000, 10000]
    MT_spectrum=spectrum(name='MT')
     
    #...Apply cuts:
    
    lhco=open_lhco(file=lhco_file, cuts=basic_cuts, lhco_header=True)

    with lhco as events:   
        for N, event in events:

            Category_τhad.start_cutflow(name='τhad')

            if event.nleps==0:
                Category_τhad.apply_cut('nleps = 0')

                if event.ntaus>=1:
                    Category_τhad.apply_cut('ntaus >= 1')
                    τ=event.tau_leading
                    MET=event.met.pt
                    
                    if MET>150.0:
                        Category_τhad.apply_cut('met > 150')

                        if abs(τ.dphi(event.met))>2.4:
                            Category_τhad.apply_cut('|dphi(τ,met)| > 2.4') 

                            if 0.7<τ.pt/MET<1.3:
                                Category_τhad.apply_cut('0.7 < pT/met < 1.3')
                                mt=τ.MT(event.met)
                                
                                MT_spectrum.extract_spectrum(mt)

                                if mt > mT_min:
                                    Category_τhad.apply_cut('mT > '+str(mT_min))

                                    lhco.parameters['signal_category']=Category_τhad
        lhco.parameters[MT_spectrum.name]=MT_spectrum
        lhco.parameters['MT_bins']=np.array(bins)
        lhco.parameters['effs']=MT_spectrum.bin_data(bins)/float(lhco.parameters['nevents'])

    return lhco.parameters


In [3]:
def extract_results(data_files):

    FF={'FS':'scalar','FT':'tensor','FST':'scalar-tensor',
        'FVLL':'vector LL','FVLR':'vector LR','FDq':'dipole'}      
    
    kernel={}
    
    for lhco in glob.glob(data_files, recursive=True):        
        print('analyzing: '+lhco)
        res=MonoTau_ATLAS_search(lhco, mT_min=200)
        info=res['run_tag'].split('_') 
        if 'int' in info: typ='SM*NP'
        else: typ='NP*NP' 
        if '00' in info:
            qqbar=res['run_tag'].split('_00_')[1].split('_')[0]+'~'
            
        elif '10' in info:
            qqbar=res['run_tag'].split('_10_')[1].split('_')[0]+'~'  
            
        elif '01' in info:
            qqbar=res['run_tag'].split('_01_')[1].split('_')[0]+'~'
        
        kernel[res['run_tag']]={}        
        kernel[res['run_tag']]['Form_Factor']=FF[info[0]]
        kernel[res['run_tag']]['FF_coeff']=list(info[1])
        kernel[res['run_tag']]['qq~']=qqbar
        kernel[res['run_tag']]['ll~']=res['run_tag'].split('_00_')[1].split('_')[1]
        kernel[res['run_tag']]['type']=typ
        kernel[res['run_tag']]['pT_bin']=[float(info[-2]),float(info[-1])]
        kernel[res['run_tag']]['results']=res
    
    return kernel


In [4]:
t0 = time.time()

FS_kernel  = extract_results(data_files='lhco_files/lhco_FS/*.gz')
FST_kernel = extract_results(data_files='lhco_files/lhco_FST/*.gz')
FT_kernel  = extract_results(data_files='lhco_files/lhco_FT/*.gz')
FVLL_kernel  = extract_results(data_files='lhco_files/lhco_FVLL/*.gz')
FD_kernel = extract_results(data_files='lhco_files/lhco_FD/*.gz')

t1 = time.time()

print(t1-t0)

analyzing: lhco_files/lhco_FS/FS_00_cb_tanu_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_cd_tanu_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_cb_tanu_1750_2000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_ub_tanu_400_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_ud_tanu_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_su_tanu_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_ub_tanu_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_su_tanu_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_bu_tanu_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_cb_tanu_400_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_us_tanu_1400_1500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_ud_tanu_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_dc_tanu_1200_1300_delphes_events.lhco.gz
analyzing: lhc

analyzing: lhco_files/lhco_FS/FS_00_sc_tanu_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_ud_tanu_1500_1750_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_cs_tanu_1750_2000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_sc_tanu_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_ub_tanu_1500_1750_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_cs_tanu_200_300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_us_tanu_200_300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_ud_tanu_1400_1500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_du_tanu_1200_1300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_ub_tanu_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_sc_tanu_1500_1750_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_sc_tanu_400_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_bc_tanu_400_500_delphes_events.lhco.gz
analyz

analyzing: lhco_files/lhco_FST/FST_00_bc_tanu_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_ub_tanu_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cd_tanu_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_bc_tanu_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cs_tanu_300_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cb_tanu_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cb_tanu_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cs_tanu_1200_1300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_us_tanu_1300_1400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_su_tanu_1750_2000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_du_tanu_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_ub_tanu_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_du_tanu_200_300_delp

analyzing: lhco_files/lhco_FST/FST_00_ud_tanu_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_ub_tanu_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_bu_tanu_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_ub_tanu_1300_1400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cb_tanu_1200_1300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cs_tanu_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_du_tanu_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_sc_tanu_1400_1500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_us_tanu_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cb_tanu_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cd_tanu_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_bc_tanu_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cs_tanu_400_500_d

analyzing: lhco_files/lhco_FT/FT_00_dc_tanu_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FT/FT_00_bc_tanu_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FT/FT_00_sc_tanu_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FT/FT_00_dc_tanu_300_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FT/FT_00_ud_tanu_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FT/FT_00_ub_tanu_1400_1500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FT/FT_00_bc_tanu_1500_1750_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FT/FT_00_du_tanu_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FT/FT_00_ud_tanu_1400_1500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FT/FT_00_du_tanu_1200_1300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FT/FT_00_sc_tanu_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FT/FT_00_bc_tanu_1400_1500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FT/FT_00_ub_tanu_1500_1750_delphes_events.lhco.gz
ana

...

analyzing: lhco_files/lhco_FVLL/FVLL_00_bu_tanu_int_1400_1500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_us_tanu_int_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_du_tanu_1400_1500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_bc_tanu_int_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_cb_tanu_int_300_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_ud_tanu_1200_1300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_dc_tanu_int_1500_1750_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_cd_tanu_1300_1400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_su_tanu_1500_1750_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_sc_tanu_int_1500_1750_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_cs_tanu_int_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_dc_tanu_200_300_delphes_events.lhco.gz
analyzing:

analyzing: lhco_files/lhco_FVLL/FVLL_00_cs_tanu_500_600_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_dc_tanu_int_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_cd_tanu_1400_1500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_su_tanu_int_400_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_du_tanu_int_200_300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_sc_tanu_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_us_tanu_500_600_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_sc_tanu_int_1200_1300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_dc_tanu_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_dc_tanu_400_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_cd_tanu_int_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_cb_tanu_int_500_600_delphes_events.lhco.gz
analyzing: lhco_f

analyzing: lhco_files/lhco_FVLL/FVLL_00_dc_tanu_int_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_sc_tanu_int_200_300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_dc_tanu_int_400_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_ud_tanu_1100_1200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_su_tanu_int_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_ub_tanu_int_1750_2000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_su_tanu_int_1500_1750_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_dc_tanu_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_ub_tanu_int_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_us_tanu_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_du_tanu_int_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_cd_tanu_int_500_600_delphes_events.lhco.gz
analy

analyzing: lhco_files/lhco_FVLL/FVLL_00_cs_tanu_int_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_dc_tanu_int_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_us_tanu_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_du_tanu_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_ub_tanu_1750_2000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_su_tanu_2000_10000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_dc_tanu_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_ub_tanu_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_us_tanu_300_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_cs_tanu_1400_1500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_dc_tanu_1300_1400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_su_tanu_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD

analyzing: lhco_files/lhco_FD/FDq_WW_00_dc_tanu_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_su_tanu_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_bu_tanu_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_cd_tanu_400_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_dc_tanu_1000_1100_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_ud_tanu_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_ub_tanu_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_ud_tanu_400_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_cb_tanu_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_su_tanu_400_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_bu_tanu_400_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_cd_tanu_1500_1750_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_WW_00_us_tanu_

In [9]:
def kernel_matrix(kernel,bins):  
    
    def get_bins(bins):
        d={}
        for i,b in enumerate(bins[:-1]):
            I=str([float(b),float(bins[i+1])])
            d[I]=i
        return d

    runs=[]    
    for run in kernel.keys():
        x='_'.join(kernel[run]['results']['run_tag'].split('_')[:-2])
        if x not in runs:
            runs.append(x)   
    dic={}
    for key in runs:
        dic[key]={}
    
    bin_number=get_bins(bins)
     
    for r in runs:
        for run in kernel.keys():
            x='_'.join(kernel[run]['results']['run_tag'].split('_')[:-2])
            b=str(kernel[run]['pT_bin'])
            eff=kernel[run]['results']['effs']
            eff_str=''
            for y in eff: eff_str+=str(y)+'\t'
            dic[x][bin_number[b]]=eff_str
                       
    for d in dic.keys():
        file=open('results/'+d,'w')
        for i in list(bin_number.values()):
            try:
                file.write(dic[d][i]+'\n')
            except KeyError:
                inv_bin_number = {v: k for k, v in bin_number.items()}
                print('Error -->',d,':',inv_bin_number[i])
                file.write((str(None)+'\t')*len(bin_number)+'\n')
        file.close()


In [12]:
bins=[200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1750, 2000, 10000]

kernel_matrix(FS_kernel,bins)
kernel_matrix(FT_kernel,bins)
kernel_matrix(FST_kernel,bins)
kernel_matrix(FVLL_kernel,bins)
kernel_matrix(FD_kernel,bins)

Error --> FS_00_cb_tanu : [500.0, 600.0]
Error --> FS_00_cb_tanu : [1400.0, 1500.0]
Error --> FS_00_cd_tanu : [1400.0, 1500.0]
Error --> FS_00_bu_tanu : [1200.0, 1300.0]
Error --> FS_00_bu_tanu : [1400.0, 1500.0]
Error --> FS_00_bu_tanu : [1750.0, 2000.0]
Error --> FS_00_dc_tanu : [1400.0, 1500.0]
Error --> FS_00_du_tanu : [1400.0, 1500.0]
Error --> FS_00_bc_tanu : [500.0, 600.0]
Error --> FS_00_bc_tanu : [1200.0, 1300.0]
Error --> FS_00_bc_tanu : [1300.0, 1400.0]
Error --> FS_00_bc_tanu : [1400.0, 1500.0]
Error --> FS_00_bc_tanu : [1500.0, 1750.0]
Error --> FS_00_bc_tanu : [1750.0, 2000.0]
Error --> FS_00_cs_tanu : [500.0, 600.0]
Error --> FS_00_cs_tanu : [1400.0, 1500.0]
Error --> FST_00_cb_tanu : [500.0, 600.0]
Error --> FVLL_00_bc_tanu_int : [1300.0, 1400.0]
Error --> FVLL_00_cb_tanu : [1300.0, 1400.0]
Error --> FVLL_00_sc_tanu_int : [1300.0, 1400.0]
Error --> FVLL_00_dc_tanu : [1300.0, 1400.0]
Error --> FVLL_00_cs_tanu_int : [1300.0, 1400.0]
Error --> FVLL_00_sc_tanu : [1300.0, 14